# 데이터 사이언스 100번의 노크(구조화 데이터 처리편) - R



## 참고(Reference) : 「データサイエンティスト協会スキル定義委員」の「データサイエンス100本ノック（構造化データ加工編）」

## 데이터 사이언티스트 소사이어티 깃허브 : https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess

## 데이터 사이언스 100 노크 (구조화된 데이터 처리) URL : https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess/blob/master/docker/work/answer/ans_preprocess_knock_R.ipynb

- 참고: 원래 데이터 사이언티스트 소사이어티(データサイエンティスト協会スキル定義委員)에서 만든 ipynb 파일을 DeepL 에서 일본어에서 영어로 번역한 파일입니다.

- 이 파일을 업데이트한 이유는 초보자부터 고급 엔지니어까지 R을 사용하고자 하는 모든 사람에게 유용한 이 실습을 전파하기 위해서입니다. 

- 이 데이터는 일본어로 작성되었으므로 연습할 때 언어 문제가 발생할 수 있습니다. 그러나 크게 영향을 미치지 않으므로 걱정하지 마십시오.

## 소개
- 먼저 다음 셀을 실행합니다.
- 필요한 라이브러리를 가져오고 데이터베이스에서 데이터를 읽습니다(PostgreSQL).
- 사용할 것으로 예상되는 라이브러리는 다음 셀에서 가져옵니다.
- 사용하려는 다른 라이브러리가 있는 경우 install.packages()를 사용하여 적절하게 설치합니다.
- 이름, 주소 등은 더미 데이터이며 실제 데이터가 아닙니다.

In [ ]:
require("RPostgreSQL")
require("tidyr")
require("dplyr")
require("stringr")
require("caret")
require("lubridate")
require("rsample")
require("recipes")
require("themis")

host <- "db"
port <- Sys.getenv()["PG_PORT"]
dbname <- Sys.getenv()["PG_DATABASE"]
user <- Sys.getenv()["PG_USER"]
password <- Sys.getenv()["PG_PASSWORD"]

con <- dbConnect(PostgreSQL(), host=host, port=port, dbname=dbname, user=user, password=password)
df_customer <- dbGetQuery(con,"SELECT * FROM customer")
df_category <- dbGetQuery(con,"SELECT * FROM category")
df_product <- dbGetQuery(con,"SELECT * FROM product")
df_receipt <- dbGetQuery(con,"SELECT * FROM receipt")
df_store <- dbGetQuery(con,"SELECT * FROM store")
df_geocode <- dbGetQuery(con,"SELECT * FROM geocode")

Loading required package: RPostgreSQL

Loading required package: DBI

Loading required package: tidyr

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: stringr

Loading required package: caret

Loading required package: ggplot2

Loading required package: lattice

Loading required package: lubridate


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loading required package: rsample

Loading required package: recipes


Attaching package: ‘recipes’


The following object is masked from ‘package:stringr’:

    fixed


The following object is masked from ‘package:stats’:

    step


Loading required package: themis

Registered S3 methods overwritten by 'themis':
  method                  from   
  bake.step

# 연습 문제

---
> R-001: 영수증 내역 데이터(df_receipt)에서 전체 항목 중 첫 10건을 표시하여 어떤 데이터를 보유하고 있는지 목시으로 확인하라.

In [ ]:
head(df_receipt, n = 10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>
1,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
2,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
3,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
4,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
5,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
6,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
7,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
8,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
9,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770


---
> R-002: 영수증 내역 데이터(df_receipt)에서 판매일자(sales_ymd), 고객 아이디(customer_id), 상품코드(product_cd), 판매금액(amount)의 순서로 열을 지정하여 10건씩 표시하라.

In [ ]:
head(df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")], n = 10)

,sales_ymd,customer_id,product_cd,amount
,<int>,<chr>,<chr>,<int>
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


---
> R-003: 영수증 내역 데이터(df_receipt)에서 매출일자(sales_ymd), 고객 ID(customer_id), 상품코드(product_cd), 매출금액(amount) 순으로 열을 지정하여 10건씩 표시한다. 단, sales_ymdsales_date로 항목명을 변경하면서 추출한다.

In [ ]:
head(rename(df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")], 
            sales_date = sales_ymd), n = 10)

,sales_date,customer_id,product_cd,amount
,<int>,<chr>,<chr>,<int>
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


---
> R-004: 영수증 내역 데이터(df_receipt)에서 판매일(sales_ymd), 고객 ID(customer_id), 상품 코드(product_cd), 판매 금액(amount) 순으로 열을 지정하고, 다음 조건을 만족하는 데이터를 추출하시오.
> - 고객 ID(customer_id)가 "CS018205000001".

In [ ]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")] %>%
    filter(customer_id == "CS018205000001")

sales_ymd,customer_id,product_cd,amount
<int>,<chr>,<chr>,<int>
20180911,CS018205000001,P071401012,2200
20180414,CS018205000001,P060104007,600
20170614,CS018205000001,P050206001,990
20170614,CS018205000001,P060702015,108
20190216,CS018205000001,P071005024,102
20180414,CS018205000001,P071101002,278
20190226,CS018205000001,P070902035,168
20190924,CS018205000001,P060805001,495
20190226,CS018205000001,P071401020,2200


---
> R-005: 영수증 내역 데이터(df_receipt)에서 판매일(sales_ymd), 고객 ID(customer_id), 상품 코드(product_cd), 판매 금액(amount)의 순서로 열을 지정하고, 아래 조건을 모두 만족하는 데이터를 추출하라.
> - 고객 ID(customer_id)가 "CS018205000001"
> - 매출 금액(amount)이 1,000 이상

In [ ]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")] %>%
    filter(customer_id == "CS018205000001" & amount >= 1000)

sales_ymd,customer_id,product_cd,amount
<int>,<chr>,<chr>,<int>
20180911,CS018205000001,P071401012,2200
20190226,CS018205000001,P071401020,2200
20180911,CS018205000001,P071401005,1100


---
> R-006: 영수증 내역 데이터(df_receipt)에서 판매일(sales_ymd), 고객 ID(customer_id), 상품 코드(product_cd), 판매 수량(quantity), 판매 금액(amount) 순으로 열을 지정하고, 다음 조건을 모두 만족하는 데이터를 추출하라. 추출하라.
> - 고객 ID(customer_id)가 "CS018205000001".
> - 판매금액(amount)이 1,000 이상 또는 판매수량(quantity)이 5 이상

In [ ]:
df_receipt[c("sales_ymd", "customer_id", 
             "product_cd", "quantity", "amount")] %>%
    filter(customer_id == "CS018205000001" & (amount >= 1000 | quantity >= 5))

sales_ymd,customer_id,product_cd,quantity,amount
<int>,<chr>,<chr>,<int>,<int>
20180911,CS018205000001,P071401012,1,2200
20180414,CS018205000001,P060104007,6,600
20170614,CS018205000001,P050206001,5,990
20190226,CS018205000001,P071401020,1,2200
20180911,CS018205000001,P071401005,1,1100


---
> R-007: 영수증 내역 데이터(df_receipt)에서 판매일(sales_ymd), 고객 ID(customer_id), 상품 코드(product_cd), 판매 금액(amount)의 순서로 열을 지정하고, 아래 조건을 모두 만족하는 데이터를 추출하라.
> - 고객 ID(customer_id)가 "CS018205000001"
> -  매출 금액(amount)이 1,000 이상 2,000 이하

In [ ]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")] %>%
    filter(customer_id == "CS018205000001" & between(amount, 1000, 2000))

sales_ymd,customer_id,product_cd,amount
<int>,<chr>,<chr>,<int>
20180911,CS018205000001,P071401005,1100


---
> R-008: 영수증 내역 데이터(df_receipt)에서 판매일(sales_ymd), 고객 ID(customer_id), 상품 코드(product_cd), 판매 금액(amount)의 순서로 열을 지정하고, 아래 조건을 모두 만족하는 데이터를 추출하라.
> - 고객 ID(customer_id)가 "CS018205000001"
> - 상품코드(product_cd)가 "P071401019" 이외

In [ ]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")] %>%
    filter(customer_id == "CS018205000001" &
           product_cd != "P071401019")

sales_ymd,customer_id,product_cd,amount
<int>,<chr>,<chr>,<int>
20180911,CS018205000001,P071401012,2200
20180414,CS018205000001,P060104007,600
20170614,CS018205000001,P050206001,990
20170614,CS018205000001,P060702015,108
20190216,CS018205000001,P071005024,102
20180414,CS018205000001,P071101002,278
20190226,CS018205000001,P070902035,168
20190924,CS018205000001,P060805001,495
20190226,CS018205000001,P071401020,2200


---
> R-009: 다음 처리에서 출력 결과를 바꾸지 않고 OR을 AND로 다시 작성하시오.

`df_store %>% filter(!(prefecture_cd == "13" | floor_area > 900))`

In [ ]:
df_store %>% filter(prefecture_cd != "13" & floor_area <= 900)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890
S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808


---
> R-010: 매장 데이터(df_store)에서 매장 코드(store_cd)가 "S14"로 시작하는 항목만 전체 추출하여 10개를 표시하라.

In [ ]:
head(df_store %>% filter(startsWith(store_cd, "S14")), n = 10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
2,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
3,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679
4,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548
5,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830
6,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574
7,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341
8,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831
9,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838


---
> R-011: 고객 데이터(df_customer)에서 고객 ID(customer_id)의 끝자리가 1인 항목만 전체 추출하여 10건을 표시하라.

In [ ]:
head(df_customer %>% filter(endsWith(customer_id, "1")), n = 10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
3,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
4,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
5,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
6,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
7,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
8,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
9,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0


---
> R-012: 점포 데이터(df_store)에서 주소(address)에 '요코하마시'가 포함된 항목만 모두 표시한다.

In [ ]:
df_store %>% filter(grepl("横浜市", address))

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890
S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657


---
> R-013: 고객 데이터(df_customer)에서 상태코드(status_cd)가 알파벳 A~F로 시작하는 데이터를 모두 추출하여 10건을 표시하라.

In [ ]:
head(df_customer %>% filter(grepl("^[A-F]", status_cd)), n = 10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
2,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
5,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
6,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
7,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
8,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
9,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E


---
> R-014: 고객 데이터(df_customer)에서 상태코드(status_cd)의 끝자리가 숫자 1~9로 끝나는 데이터를 모두 추출하여 10건을 표시하라.

In [ ]:
head(df_customer %>% filter(grepl("[1-9]$", status_cd)), n = 10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
2,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
5,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
6,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
7,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
8,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
9,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1


---
> R-015: 고객 데이터(df_customer)에서 상태코드(status_cd)의 시작이 알파벳 A~F로 시작하고 끝이 숫자 1~9로 끝나는 데이터를 모두 추출하여 10건을 표시하라.

In [ ]:
head(df_customer %>% filter(grepl("^[A-F].*[1-9]$", status_cd)), n = 10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
2,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
3,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
4,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
5,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
6,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
7,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
8,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
9,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1


---
> R-016: 매장 데이터(df_store)에서 전화번호(tel_no)가 3자리-3자리-4자리인 데이터를 모두 표시하라.

In [ ]:
df_store %>% filter(grepl("^[0-9]{3}-[0-9]{3}-[0-9]{4}$", tel_no))

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830
S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574


---
> R-017: 고객 데이터(df_customer)를 생년월일(birth_day)을 기준으로 나이순으로 정렬하고, 맨 위부터 10개의 항목을 모두 표시하라.

In [ ]:
head(
    df_customer[order(df_customer$birth_day), ],
    n = 10
)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
18818,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12329,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15683,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15303,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1682,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7512,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2379,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4681,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
6306,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> R-018: 고객 데이터(df_customer)를 생년월일(birth_day)을 기준으로 젊은 순서로 정렬하고, 맨 위부터 10개의 항목을 모두 표시하라.

In [ ]:
head(
    df_customer[order(df_customer$birth_day,
                      decreasing = TRUE), ],
    n = 10
)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
15640,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7469,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10746,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19812,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7040,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3671,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12494,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15978,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5717,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F


---
> R-019: 영수증 내역 데이터(df_receipt)에 대해 건당 매출 금액(amount)이 높은 순서대로 순위를 부여하고, 맨 위부터 10건씩 표시한다. 항목은 고객 ID(customer_id), 매출 금액(amount), 부여한 랭크를 표시하도록 한다. 단, 매출액(amount)이 동일한 경우 동일한 순위를 부여한다.

In [ ]:
df_receipt[c("customer_id", "amount")] %>%
    mutate(ranking = min_rank(desc(amount))) %>%
    arrange(ranking) %>%
    slice(1:10)

customer_id,amount,ranking
<chr>,<int>,<int>
CS011415000006,10925,1
ZZ000000000000,6800,2
CS028605000002,5780,3
CS015515000034,5480,4
ZZ000000000000,5480,4
ZZ000000000000,5480,4
ZZ000000000000,5440,7
CS021515000089,5440,7
CS015515000083,5280,9


---
> R-020: 영수증 내역 데이터(df_receipt)에 대해 건당 매출 금액(amount)이 높은 순서대로 순위를 부여하고, 맨 위부터 10건씩 표시한다. 항목은 고객 ID(customer_id), 매출 금액(amount), 부여한 순위를 표시한다. 단, 매출금액(amount)이 동일한 경우에도 별도의 순위를 부여해야 한다.

In [ ]:
df_receipt[c("customer_id", "amount")] %>%
    mutate(ranking = row_number(desc(amount))) %>%
    arrange(ranking) %>%
    slice(1:10) 

customer_id,amount,ranking
<chr>,<int>,<int>
CS011415000006,10925,1
ZZ000000000000,6800,2
CS028605000002,5780,3
CS015515000034,5480,4
ZZ000000000000,5480,5
ZZ000000000000,5480,6
ZZ000000000000,5440,7
CS021515000089,5440,8
CS015515000083,5280,9


---
> R-021: 영수증 내역 데이터(df_receipt)에 대해 건수를 세어라.

In [ ]:
nrow(df_receipt)

[1] 104681

---
> R-022: 영수증 내역 데이터(df_receipt)의 고객 ID(customer_id)에 대해 고유 건수를 세어라.

In [ ]:
length(unique(df_receipt$customer_id))

[1] 8307

---
> R-023: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별 판매 금액(amount)과 판매 수량(quantity)을 합산하라.

In [ ]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(amount = sum(amount),
              quantity = sum(quantity), .groups = "drop")

store_cd,amount,quantity
<chr>,<int>,<int>
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403
S13001,811936,2347
S13002,727821,2340
S13003,764294,2197
S13004,779373,2390


---
> R-024: 영수증 내역 데이터(df_receipt)에 대해 고객 ID(customer_id)별로 가장 최근 매출 날짜(sales_ymd)를 구하여 10건 표시하라.

In [ ]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(max_ymd = max(sales_ymd), .groups = "drop") %>%
    slice(1:10)

customer_id,max_ymd
<chr>,<int>
CS001113000004,20190308
CS001114000005,20190731
CS001115000010,20190405
CS001205000004,20190625
CS001205000006,20190224
CS001211000025,20190322
CS001212000027,20170127
CS001212000031,20180906
CS001212000046,20170811


---
> R-025: 영수증 내역 데이터(df_receipt)에 대해 고객 ID(customer_id)별로 가장 오래된 매출 날짜(sales_ymd)를 구하여 10건 표시하라.

In [ ]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(min_ymd = min(sales_ymd), .groups = "drop") %>%
    slice(1:10)

customer_id,min_ymd
<chr>,<int>
CS001113000004,20190308
CS001114000005,20180503
CS001115000010,20171228
CS001205000004,20170914
CS001205000006,20180207
CS001211000025,20190322
CS001212000027,20170127
CS001212000031,20180906
CS001212000046,20170811


---
> R-026: 영수증 내역 데이터(df_receipt)에 대해 고객 ID(customer_id)별로 가장 최근 매출 날짜(sales_ymd)와 가장 오래된 매출 날짜(sales_ymd)를 구하고, 양자가 서로 다른 데이터 10건을 표시하라.

In [ ]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(max_ymd = max(sales_ymd)
             ,min_ymd = min(sales_ymd), .groups = "drop") %>%
    filter(max_ymd != min_ymd) %>%
    slice(1:10)

customer_id,max_ymd,min_ymd
<chr>,<int>,<int>
CS001114000005,20190731,20180503
CS001115000010,20190405,20171228
CS001205000004,20190625,20170914
CS001205000006,20190224,20180207
CS001214000009,20190902,20170306
CS001214000017,20191006,20180828
CS001214000048,20190929,20171109
CS001214000052,20190617,20180208
CS001215000005,20181021,20170206


---
> R-027: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별 매출 금액(amount)의 평균을 계산하여 내림차순으로 TOP5를 표시하라.

In [ ]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(mean_amount = mean(amount), .groups = "drop") %>%
    arrange(desc(mean_amount)) %>%
    slice(1:5)

store_cd,mean_amount
<chr>,<dbl>
S13052,402.8675
S13015,351.1120
S13003,350.9155
S14010,348.7913
S13001,348.4704


---
> R-028: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별 매출 금액(amount)의 중앙값을 계산하여 내림차순으로 TOP5를 표시하라.

In [ ]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(median_amount = median(amount), .groups = "drop") %>%
    arrange(desc(median_amount)) %>%
    slice(1:5)

store_cd,median_amount
<chr>,<dbl>
S13052,190
S14010,188
S14050,185
S13003,180
S13018,180


---
> R-029: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별로 상품 코드(product_cd)의 최빈값을 구하여 10건씩 표시하라.

In [ ]:
# 코드 예시 1
df_receipt %>%
  group_by(store_cd, product_cd) %>%
  summarise(cnt = n(), .groups = "drop_last") %>%
  filter(cnt == cnt %>% max()) %>%
  ungroup() %>%
  slice(1:10)

store_cd,product_cd,cnt
<chr>,<chr>,<int>
S12007,P060303001,72
S12013,P060303001,107
S12014,P060303001,65
S12029,P060303001,92
S12030,P060303001,115
S13001,P060303001,67
S13002,P060303001,78
S13003,P071401001,65
S13004,P060303001,88


In [ ]:
# 코드 예시 2: which.max() 사용 (가장 빈번한 값이 여러 개 발생 시, 어느 한 개로 좁혀짐)
# 건수 표시 생략

table_product <-table(df_receipt$store_cd, df_receipt$product_cd)

store <- names(table_product[,1])

mode_product <- c()

for (i in 1:length(store)){
    mode_product[i] <- names(which.max(table_product[i,]))
}

data.frame(store_cd = store, product_cd = mode_product) %>%
  slice(1:10)

store_cd,product_cd
<chr>,<chr>
S12007,P060303001
S12013,P060303001
S12014,P060303001
S12029,P060303001
S12030,P060303001
S13001,P060303001
S13002,P060303001
S13003,P071401001
S13004,P060303001


---
> R-030: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별 매출 금액(amount)의 분산을 계산하여 내림차순으로 5개씩 표시하시오.

In [ ]:
var_sample <- function(x){ var(x) * (length(x) - 1) / length(x) }

df_receipt %>%
    group_by(store_cd) %>%
    summarise(var_amount = var_sample(amount), .groups = "drop") %>%
    arrange(desc(var_amount)) %>%
    slice(1:5)

store_cd,var_amount
<chr>,<dbl>
S13052,440088.7
S14011,306314.6
S14034,296920.1
S13001,295432.0
S13015,295294.4


---
> R-031: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별 매출 금액(amount)의 표준편차를 계산하여 내림차순으로 5건씩 표시하라.

In [ ]:
var_sample <- function(x){ var(x)*(length(x)-1)/length(x) }

std_sample <- function(x){ sqrt(var_sample(x)) }

head(
    df_receipt %>%
        group_by(store_cd) %>%
        summarise(std_amount = std_sample(amount), .groups = "drop") %>%
        arrange(desc(std_amount))
    , n = 5
)

store_cd,std_amount
<chr>,<dbl>
S13052,663.3918
S14011,553.4569
S14034,544.9037
S13001,543.5366
S13015,543.4099


---
> R-032: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)에 대해 25% 단위로 백분위수 값을 구하라.

In [ ]:
df_receipt %>%
    summarise(amount_25per = quantile(amount, 0.25),
              amount_50per = quantile(amount, 0.5),
              amount_75per = quantile(amount, 0.75),
              amount_100per = quantile(amount, 1.0))

amount_25per,amount_50per,amount_75per,amount_100per
<dbl>,<dbl>,<dbl>,<dbl>
102,170,288,10925


---
> R-033: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별 매출 금액(amount)의 평균을 계산하여 330개 이상을 추출하라.

In [ ]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(mean_amount = mean(amount), .groups = "drop") %>%
    filter(mean_amount >= 330)

store_cd,mean_amount
<chr>,<dbl>
S12013,330.1941
S13001,348.4704
S13003,350.9155
S13004,330.9439
S13015,351.1120
S13019,330.2086
S13020,337.8799
S13052,402.8675
S14010,348.7913


---
> R-034: 영수증 내역 데이터(df_receipt)에 대해 고객 ID(customer_id)별로 매출 금액(amount)을 합산하여 전체 고객의 평균을 구하시오. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

In [ ]:
df_mean <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    summarise(mean_amount = mean(sum_amount))

df_mean$mean_amount

[1] 2547.742

---
> R-035: 영수증 내역 데이터(df_receipt)에 대해 고객 ID(customer_id)별로 매출 금액(amount)을 합산하여 전체 고객의 평균을 구하고, 평균 이상 쇼핑을 한 고객을 추출하여 10건씩 표시하시오. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

In [ ]:
df_sum <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

df_mean <- df_sum %>%
    summarise(mean_amount = mean(sum_amount))

df_sum %>%
    filter(sum_amount >= df_mean$mean_amount) %>%
    slice(1:10)

customer_id,sum_amount
<chr>,<int>
CS001115000010,3044
CS001205000006,3337
CS001214000009,4685
CS001214000017,4132
CS001214000052,5639
CS001215000040,3496
CS001304000006,3726
CS001305000005,3485
CS001305000011,4370


---
> R-036: 영수증 내역 데이터(df_receipt)와 매장 데이터(df_store)를 내부적으로 결합하여 영수증 내역 데이터의 모든 항목과 매장 데이터의 매장명(store_name)을 10건씩 표시하라.

In [ ]:
inner_join(df_receipt, 
           df_store[c("store_cd", "store_name")], by = "store_cd") %>%
    slice(1:10)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>
20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81,成城店
20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170,二ツ橋店
20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25,新山下店
20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90,大和店
20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138,狛江店
20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30,三田店
20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128,長津田店
20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770,十条仲原店


---
> R-037: 상품 데이터(df_product)와 카테고리 데이터(df_category)를 내부적으로 결합하여 상품 데이터의 전체 항목과 카테고리 데이터의 카테고리 소분류 이름(category_small_name) 10개를 표시하라.

In [ ]:
inner_join(df_product,
           df_category[c("category_small_cd", "category_small_name")],
           by = "category_small_cd") %>%
    slice(1:10)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>
P040101001,04,0401,040101,198,149,弁当類
P040101002,04,0401,040101,218,164,弁当類
P040101003,04,0401,040101,230,173,弁当類
P040101004,04,0401,040101,248,186,弁当類
P040101005,04,0401,040101,268,201,弁当類
P040101006,04,0401,040101,298,224,弁当類
P040101007,04,0401,040101,338,254,弁当類
P040101008,04,0401,040101,420,315,弁当類
P040101009,04,0401,040101,498,374,弁当類


---
> R-038: 고객 데이터(df_customer)와 영수증 내역 데이터(df_receipt)에서 고객별 매출 금액의 총합을 구하여 10건을 표시하시오. 단, 판매 실적이 없는 고객에 대해서는 판매금액을 0으로 표시한다. 또한, 고객은 성별 코드(gender_cd)가 여성(1)인 고객을 대상으로 하며, 비회원(고객 ID가 "Z"로 시작하는 고객)은 제외한다.

In [ ]:
df_sum <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

df_target <- df_customer %>%
    filter(gender_cd == "1" & !grepl("^Z", customer_id))

left_join(df_target["customer_id"], df_sum, by = "customer_id") %>%
    replace_na(list(sum_amount = 0)) %>%
    slice(1:10)

customer_id,sum_amount
<chr>,<dbl>
CS021313000114,0
CS031415000172,5088
CS028811000001,0
CS001215000145,875
CS015414000103,3122
CS033513000180,868
CS035614000014,0
CS011215000048,3444
CS009413000079,0


---
> R-039: 영수증 내역 데이터(df_receipt)에서 매출 일수가 많은 고객 상위 20건을 추출한 데이터와 매출 금액 합계가 많은 고객 상위 20건을 추출한 데이터를 각각 생성하고, 이 두 가지를 완전히 외부 결합하라. 단, 비회원(고객 ID가 "Z"로 시작하는 고객)은 제외한다.

In [ ]:
# 코그 예시 1
df_data <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id)

df_cnt <- df_data %>%
    summarise(come_days = n_distinct(sales_ymd), .groups = "drop") %>%
    arrange(desc(come_days), customer_id) %>%
    slice(1:20)

df_sum <- df_data %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    arrange(desc(sum_amount)) %>%
    slice(1:20)

full_join(df_cnt, df_sum, by = "customer_id")

customer_id,come_days,sum_amount
<chr>,<int>,<int>
CS040214000008,23,NA
CS010214000010,22,18585
CS015415000185,22,20153
CS010214000002,21,NA
CS028415000007,21,19127
CS016415000141,20,18372
CS017415000097,20,23086
CS014214000023,19,NA
CS021514000045,19,NA


In [ ]:
# 코그 예시 2
# slice_max를 사용한 코드 예시.
# with_ties = FALSE를 with_ties = TRUE로 다시 작성하여 동일한 등급을 포함하도록 양식을 변경합니다.

df_data <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id)

df_cnt <- df_data %>%
    summarise(come_days = n_distinct(sales_ymd)) %>%
    slice_max(come_days, n = 20, with_ties = FALSE)

df_sum <- df_data %>%
    summarise(sum_amount = sum(amount)) %>%
    slice_max(sum_amount, n = 20, with_ties = FALSE)

full_join(df_cnt, df_sum, by = "customer_id")

customer_id,come_days,sum_amount
<chr>,<int>,<int>
CS040214000008,23,NA
CS010214000010,22,18585
CS015415000185,22,20153
CS010214000002,21,NA
CS028415000007,21,19127
CS016415000141,20,18372
CS017415000097,20,23086
CS014214000023,19,NA
CS021514000045,19,NA


---
> R-040: 모든 매장과 모든 상품을 조합한 데이터를 생성하고 싶다. 매장 데이터(df_store)와 상품 데이터(df_product)를 직적분하여 건수를 계산하라.

In [ ]:
df_store_tmp <- df_store

df_product_tmp <- df_product

df_store_tmp["key"] <- 0

df_product_tmp["key"] <- 0

nrow(full_join(df_store_tmp, df_product_tmp, by = "key"))

[1] 531590

---
> R-041: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 날짜(sales_ymd)별로 집계하여, 지난번 매출이 있었던 날로부터의 매출 금액 증감을 계산하라. 그리고 결과를 10건 표시하라.

In [ ]:
df_receipt %>%
    group_by(sales_ymd) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(lag_ymd = lag(sales_ymd), 
           lag_amount = lag(sum_amount), 
           diff_amount = sum_amount - lag_amount) %>%
    slice(1:10)

sales_ymd,sum_amount,lag_ymd,lag_amount,diff_amount
<int>,<int>,<int>,<int>,<int>
20170101,33723,NA,NA,NA
20170102,24165,20170101,33723,-9558
20170103,27503,20170102,24165,3338
20170104,36165,20170103,27503,8662
20170105,37830,20170104,36165,1665
20170106,32387,20170105,37830,-5443
20170107,23415,20170106,32387,-8972
20170108,24737,20170107,23415,1322
20170109,26718,20170108,24737,1981


---
> R-042: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 날짜(sales_ymd)별로 집계하고, 각 날짜의 데이터에 대해 이전, 전전, 3회 전에 매출이 있었던 날의 데이터를 결합하라. 그리고 결과를 10건 표시하라.

In [ ]:
# 코드 예시 1: 세로형 케이스
df_sum <-df_receipt %>%
    group_by(sales_ymd) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

for (i in 1:3) {
    df_tmp <- df_sum %>%
        mutate(lag_ymd = lag(sales_ymd, n = i), 
               lag_amount = lag(sum_amount, n = i))
    if (i == 1) {
        df_lag <- df_tmp
    }
    else {
        df_lag <- rbind(df_lag, df_tmp)
    }
}

df_lag %>%
    drop_na(everything()) %>%
    arrange(sales_ymd, lag_ymd) %>%
    slice(1:10)

sales_ymd,sum_amount,lag_ymd,lag_amount
<int>,<int>,<int>,<int>
20170102,24165,20170101,33723
20170103,27503,20170101,33723
20170103,27503,20170102,24165
20170104,36165,20170101,33723
20170104,36165,20170102,24165
20170104,36165,20170103,27503
20170105,37830,20170102,24165
20170105,37830,20170103,27503
20170105,37830,20170104,36165


In [ ]:
# 코드 예시 2: 가로형 케이스
df_sum <-df_receipt %>%
    group_by(sales_ymd) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

for (i in 1:3) {
    col_name_1 <- paste("lag_ymd_", i , sep="_")
    col_name_2 <- paste("lag_amount", i , sep="_")
    df_tmp <- df_sum %>%
        mutate(!!col_name_1 := lag(sales_ymd, n　=　i), 
               !!col_name_2 := lag(sum_amount, n　=　i))
    if (i == 1)　{
        df_lag <- df_tmp
    }
    else {
        df_lag <- cbind(df_lag, df_tmp[c(col_name_1, col_name_2)])
    }
}

df_lag %>%
    drop_na(everything()) %>%
    arrange(sales_ymd) %>%
    slice(1:10)

sales_ymd,sum_amount,lag_ymd__1,lag_amount_1,lag_ymd__2,lag_amount_2,lag_ymd__3,lag_amount_3
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
20170104,36165,20170103,27503,20170102,24165,20170101,33723
20170105,37830,20170104,36165,20170103,27503,20170102,24165
20170106,32387,20170105,37830,20170104,36165,20170103,27503
20170107,23415,20170106,32387,20170105,37830,20170104,36165
20170108,24737,20170107,23415,20170106,32387,20170105,37830
20170109,26718,20170108,24737,20170107,23415,20170106,32387
20170110,20143,20170109,26718,20170108,24737,20170107,23415
20170111,24287,20170110,20143,20170109,26718,20170108,24737
20170112,23526,20170111,24287,20170110,20143,20170109,26718


---
> R-043: 영수증 내역 데이터(receipt)와 고객 데이터(customer)를 결합하여 성별 코드(gender_cd)와 연령(age에서 계산)별 매출 금액(amount)을 합산한 매출 요약 데이터를 생성한다. 성별 코드는 0은 남성, 1은 여성, 9는 알 수 없음을 나타낸다.
>
> 단, 항목 구성은 연령, 여성 매출금액, 남성 매출금액, 여성 매출금액, 성별 미상 매출금액의 4개 항목으로 구성한다(세로로 연령, 가로로 성별 교차집계). 또한 연령은 10세 단위의 계급으로 한다.

In [ ]:
df_sales_summary <- df_customer[c("customer_id", "gender_cd", "age")] %>%
    mutate(era = trunc(age / 10) * 10) %>%
    inner_join(df_receipt, by = "customer_id") %>%
    group_by(gender_cd, era) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    spread(
        gender_cd, 
        sum_amount, 
        fill = 0) %>%
    rename(male = "0", female = "1", unknown = "9")

df_sales_summary

era,male,female,unknown
<dbl>,<dbl>,<dbl>,<dbl>
10,1591,149836,4317
20,72940,1363724,44328
30,177322,693047,50441
40,19355,9320791,483512
50,54320,6685192,342923
60,272469,987741,71418
70,13435,29764,2427
80,46360,262923,5111
90,0,6260,0


---
> R-044: 043에서 만든 매출 요약 데이터(df_sales_summary)는 성별 매출을 가로로 나열한 데이터였다. 이 데이터에서 성별을 세로로 가져와서 연령, 성별 코드, 매출 금액의 세 가지 항목으로 변환하라. 단, 성별 코드는 남성을 '00', 여성을 '01', 알 수 없음을 '99'로 한다.

In [ ]:
df_sales_summary　%>%
    gather(key = gender_cd, 
        value = sum_amount, male, female, unknown) %>%
    mutate(gender_cd = case_when(
        gender_cd == "male"  ~ "00",
        gender_cd == "female"  ~ "01",
        gender_cd == "unknown"  ~ "99",
    ))

era,gender_cd,sum_amount
<dbl>,<chr>,<dbl>
10,00,1591
20,00,72940
30,00,177322
40,00,19355
50,00,54320
60,00,272469
70,00,13435
80,00,46360
90,00,0


---
> R-045: 고객 데이터(df_customer)의 생년월일(birth_day)은 날짜형으로 데이터를 보유하고 있다. 이를 YYYYMMDD 형식의 문자열로 변환하여 고객 ID(customer_id)와 함께 10건 표시하라.

In [ ]:
df_tmp <- cbind(df_customer["customer_id"], 
                strftime(
                    df_customer$birth_day,
                    format = "%Y%m%d"
                ))

colnames(df_tmp) <- c("customer_id", "birth_day")

head(df_tmp,10)

,customer_id,birth_day
,<chr>,<chr>
1,CS021313000114,19810429
2,CS037613000071,19520401
3,CS031415000172,19761004
4,CS028811000001,19330327
5,CS001215000145,19950329
6,CS020401000016,19740915
7,CS015414000103,19770809
8,CS029403000008,19730817
9,CS015804000004,19310502


---
> R-046: 고객 데이터(df_customer)의 신청일(application_date)은 YYYYMMDD 형식의 문자열 형태로 데이터를 보유하고 있다. 이를 날짜형으로 변환하여 고객 ID(customer_id)와 함께 10건씩 표시하라.

In [ ]:
df_tmp <- cbind(df_customer["customer_id"], 
                strptime(df_customer$application_date, "%Y%m%d"))

colnames(df_tmp) <- c("customer_id", "application_date")

head(df_tmp, 10)

,customer_id,application_date
,<chr>,<dttm>
1,CS021313000114,2015-09-05
2,CS037613000071,2015-04-14
3,CS031415000172,2015-05-29
4,CS028811000001,2016-01-15
5,CS001215000145,2017-06-05
6,CS020401000016,2015-02-25
7,CS015414000103,2015-07-22
8,CS029403000008,2015-05-15
9,CS015804000004,2015-06-07


---
> R-047: 영수증 내역 데이터(df_receipt)의 매출일(sales_ymd)은 YYYYMMDD 형식의 숫자형으로 데이터를 보유하고 있다. 이를 날짜형으로 변환하여 영수증 번호(receipt_no), 영수증 하위번호(receipt_sub_no)와 함께 10건씩 표시하라.

In [ ]:
df_tmp <- cbind(df_receipt[c("receipt_no", "receipt_sub_no")], 
                strptime(as.character(df_receipt$sales_ymd), "%Y%m%d"))

colnames(df_tmp) <- c("receipt_no", "receipt_sub_no", "sales_ymd")

head(df_tmp, 10)

,receipt_no,receipt_sub_no,sales_ymd
,<int>,<int>,<dttm>
1,112,1,2018-11-03
2,1132,2,2018-11-18
3,1102,1,2017-07-12
4,1132,1,2019-02-05
5,1102,2,2018-08-21
6,1112,1,2019-06-05
7,1102,2,2018-12-05
8,1102,1,2019-09-22
9,1112,2,2017-05-04


---
> R-048: 영수증 내역 데이터(df_receipt)의 매출 에포크 초(sales_epoch)는 숫자형 UNIX 초로 데이터를 보유하고 있다. 이를 날짜형으로 변환하여 영수증 번호(receipt_no), 영수증 서브번호(receipt_sub_no)와 함께 10건을 표시하라.

In [ ]:
df_tmp <- cbind(df_receipt[c("receipt_no", "receipt_sub_no")], 
                as.POSIXct(df_receipt$sales_epoch, origin="1970-01-01"))

colnames(df_tmp) <- c("receipt_no", "receipt_sub_no", "sales_ymd")

head(df_tmp, 10)

,receipt_no,receipt_sub_no,sales_ymd
,<int>,<int>,<dttm>
1,112,1,2018-11-03
2,1132,2,2018-11-18
3,1102,1,2017-07-12
4,1132,1,2019-02-05
5,1102,2,2018-08-21
6,1112,1,2019-06-05
7,1102,2,2018-12-05
8,1102,1,2019-09-22
9,1112,2,2017-05-04


---
> R-049: 영수증 내역 데이터(df_receipt)의 매출 에포크 초(sales_epoch)를 날짜형으로 변환하여 '연도'만 추출하여 영수증 번호(receipt_no), 영수증 하위 번호(receipt_sub_no)와 함께 10건 표시하라.

In [ ]:
df_tmp <- cbind(df_receipt[c("receipt_no", "receipt_sub_no")], 
                substring(as.POSIXct(df_receipt$sales_epoch, 
                                     origin = "1970-01-01"), 1, 4))

colnames(df_tmp) <- c("receipt_no", "receipt_sub_no", "sales_year")

head(df_tmp, 10)

,receipt_no,receipt_sub_no,sales_year
,<int>,<int>,<chr>
1,112,1,2018
2,1132,2,2018
3,1102,1,2017
4,1132,1,2019
5,1102,2,2018
6,1112,1,2019
7,1102,2,2018
8,1102,1,2019
9,1112,2,2017


---
> R-050: 영수증 내역 데이터(df_receipt)의 매출 에포크 초(sales_epoch)를 날짜형으로 변환하여 '월'만 추출하여 영수증 번호(receipt_no), 영수증 하위 번호(receipt_sub_no)와 함께 10건을 표시한다. 단, '월'은 0으로 채워진 2자리로 추출한다.

In [ ]:
df_tmp <- cbind(df_receipt[c("receipt_no", "receipt_sub_no")], 
                substring(as.POSIXct(df_receipt$sales_epoch, 
                                     origin = "1970-01-01"), 6, 7))

colnames(df_tmp) <- c("receipt_no", "receipt_sub_no", "sales_ymd")

head(df_tmp, 10)

,receipt_no,receipt_sub_no,sales_ymd
,<int>,<int>,<chr>
1,112,1,11
2,1132,2,11
3,1102,1,07
4,1132,1,02
5,1102,2,08
6,1112,1,06
7,1102,2,12
8,1102,1,09
9,1112,2,05


---
> R-051: 영수증 내역 데이터(df_receipt)의 매출 에포크 초를 날짜형으로 변환하여 '일'만 추출하여 영수증 번호(receipt_no), 영수증 하위 번호(receipt_sub_no)와 함께 10건 표시한다. 단, '일'은 0으로 채워진 2자리로 추출한다.

In [ ]:
df_tmp <- cbind(df_receipt[c("receipt_no", "receipt_sub_no")], 
                substring(as.POSIXct(df_receipt$sales_epoch, 
                                     origin = "1970-01-01"), 9, 10))

colnames(df_tmp) <- c("receipt_no", "receipt_sub_no", "sales_day")

head(df_tmp, 10)

,receipt_no,receipt_sub_no,sales_day
,<int>,<int>,<chr>
1,112,1,03
2,1132,2,18
3,1102,1,12
4,1132,1,05
5,1102,2,21
6,1112,1,05
7,1102,2,05
8,1102,1,22
9,1112,2,04


---
> R-052: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 고객 ID(customer_id)별로 합산한 후, 매출 금액 총합에 대해 2,000원 이하를 0, 2,000원보다 큰 금액을 1로 이분화하여 고객 ID, 매출 금액 총합과 함께 10건씩 표시한다. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

In [ ]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount=sum(amount), .groups = "drop") %>%
    mutate(sales_flg = ifelse(sum_amount > 2000, 1, 0))　%>%
    slice(1:10)

customer_id,sum_amount,sales_flg
<chr>,<int>,<dbl>
CS001113000004,1298,0
CS001114000005,626,0
CS001115000010,3044,1
CS001205000004,1988,0
CS001205000006,3337,1
CS001211000025,456,0
CS001212000027,448,0
CS001212000031,296,0
CS001212000046,228,0


---
> R-053: 고객 데이터(df_customer)의 우편번호(postal_cd)에 대해 도쿄(앞 3자리가 100~209인 것)를 1, 그 외의 것을 0으로 이진화하라. 또한 영수증 내역 데이터(df_receipt)와 결합하여 전체 기간 동안 매출 실적이 있는 고객 수를 생성한 이항대수별로 계산하라.

In [ ]:
df_customer[c("customer_id", "postal_cd")] %>%
    mutate(postal_flg = 
           ifelse(100 <= as.integer(str_sub(postal_cd, start = 1, end = 3)) 
                  & as.integer(str_sub(postal_cd, start = 1, end = 3)) <= 209, 
                  1, 0)) %>%
    inner_join(df_receipt, by = "customer_id") %>%
    group_by(postal_flg) %>%
    summarise(customer_cnt = n_distinct(customer_id), .groups = "drop")

postal_flg,customer_cnt
<dbl>,<int>
0,3906
1,4400


---
> R-054: 고객 데이터(df_customer)의 주소(address)는 사이타마현, 지바현, 도쿄도, 가나가와현 중 하나이다. 도도부현별로 코드 값을 생성하여 고객 ID, 주소와 함께 10건씩 표시하라. 값은 사이타마현을 11, 지바현을 12, 도쿄도를 13, 가나가와현을 14로 한다.

In [ ]:
# 코드 예제 1(고정에서 잘라내기)
df_customer %>%
    mutate(prefecture_cd = 
           case_when(
               str_sub(address, start = 1, end = 3) == "埼玉県" ~ "11",
               str_sub(address, start = 1, end = 3) == "千葉県" ~ "12",
               str_sub(address, start = 1, end = 3) == "東京都" ~ "13",
               str_sub(address, start = 1, end = 3) == "神奈川" ~ "14")) %>%
    select(customer_id, address, prefecture_cd) %>%
    slice(1:10)

customer_id,address,prefecture_cd
<chr>,<chr>,<chr>
CS021313000114,神奈川県伊勢原市粟窪**********,14
CS037613000071,東京都江東区南砂**********,13
CS031415000172,東京都渋谷区代々木**********,13
CS028811000001,神奈川県横浜市泉区和泉町**********,14
CS001215000145,東京都大田区仲六郷**********,13
CS020401000016,東京都板橋区若木**********,13
CS015414000103,東京都江東区北砂**********,13
CS029403000008,千葉県浦安市海楽**********,12
CS015804000004,東京都江東区北砂**********,13


In [ ]:
# 코드 예제 2(정규식 사용)
df_customer %>%
  mutate(prefecture_cd = 
    case_when(
      str_extract(address, pattern = "^.*?[都道府県]") == "埼玉県" ~ "11",
      str_extract(address, pattern = "^.*?[都道府県]") == "千葉県" ~ "12",
      str_extract(address, pattern = "^.*?[都道府県]") == "東京都" ~ "13",
      str_extract(address, pattern = "^.*?[都道府県]") == "神奈川県" ~ "14")) %>%
    select(customer_id, address, prefecture_cd) %>%
    slice(1:10)

customer_id,address,prefecture_cd
<chr>,<chr>,<chr>
CS021313000114,神奈川県伊勢原市粟窪**********,14
CS037613000071,東京都江東区南砂**********,13
CS031415000172,東京都渋谷区代々木**********,13
CS028811000001,神奈川県横浜市泉区和泉町**********,14
CS001215000145,東京都大田区仲六郷**********,13
CS020401000016,東京都板橋区若木**********,13
CS015414000103,東京都江東区北砂**********,13
CS029403000008,千葉県浦安市海楽**********,12
CS015804000004,東京都江東区北砂**********,13


---
> R-055: 영수증 명세서(df_receipt) 데이터의 매출 금액(amount)을 고객 ID(customer_id)별로 합산하고, 그 합산 금액의 사분위수를 구하시오. 그 후, 고객별 매출금액 합계에 대해 아래 기준으로 카테고리 값을 생성하여 고객 ID, 매출금액 합계와 함께 10건씩 표시하라. 카테고리 값은 순서대로 1~4로 한다.
> 
> - 최소값 이상 1사분위수 미만 ・・・ 1을 부여
> - 1사분위 이상 2사분위 미만 ・・・ 2를 부여
> - 2사분위 이상 3사분위 미만 ・・・ 3을 부여
> - 3사분위 이상 ・・・ 4 부여

In [ ]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(pct_group = case_when(
        sum_amount < quantile(sum_amount)[2]  ~ "1",
        sum_amount < quantile(sum_amount)[3]  ~ "2",
        sum_amount < quantile(sum_amount)[4]  ~ "3",
        quantile(sum_amount)[4] <= sum_amount ~ "4"
    )) %>%
    slice(1:10)

customer_id,sum_amount,pct_group
<chr>,<int>,<chr>
CS001113000004,1298,2
CS001114000005,626,2
CS001115000010,3044,3
CS001205000004,1988,3
CS001205000006,3337,3
CS001211000025,456,1
CS001212000027,448,1
CS001212000031,296,1
CS001212000046,228,1


---
> R-056: 고객 데이터(df_customer)의 나이(age)를 기준으로 10세 단위로 연령을 계산하여 고객 ID(customer_id), 생년월일(birth_day)과 함께 10건씩 표시한다. 단, 60세 이상은 모두 60대 이상으로 한다. 연령을 나타내는 카테고리 명칭은 임의대로 한다.

In [ ]:
# 코드 예시 1

df_customer[c("customer_id", "birth_day", "age")] %>%
    mutate(era = trunc(age / 10) * 10) %>%
    mutate(era = case_when(
        era < 60 ~ era,
        era >= 60 ~　60
    )) %>%
    slice(1:10)

customer_id,birth_day,age,era
<chr>,<date>,<int>,<dbl>
CS021313000114,1981-04-29,37,30
CS037613000071,1952-04-01,66,60
CS031415000172,1976-10-04,42,40
CS028811000001,1933-03-27,86,60
CS001215000145,1995-03-29,24,20
CS020401000016,1974-09-15,44,40
CS015414000103,1977-08-09,41,40
CS029403000008,1973-08-17,45,40
CS015804000004,1931-05-02,87,60


In [ ]:
# 코드 예제 2(min 사용 시)

df_customer[c("customer_id", "birth_day","age")] %>%
    mutate(era = sapply(1:nrow(df_customer), 
                function(x){
                    return (min(trunc(.[x, 3] / 10) * 10, 60))
                })) %>%
    slice(1:10)

customer_id,birth_day,age,era
<chr>,<date>,<int>,<dbl>
CS021313000114,1981-04-29,37,30
CS037613000071,1952-04-01,66,60
CS031415000172,1976-10-04,42,40
CS028811000001,1933-03-27,86,60
CS001215000145,1995-03-29,24,20
CS020401000016,1974-09-15,44,40
CS015414000103,1977-08-09,41,40
CS029403000008,1973-08-17,45,40
CS015804000004,1931-05-02,87,60


---
> R-057: 056의 추출 결과와 성별 코드(gender_cd)에 따라 성별×연령의 조합을 나타내는 카테고리 데이터를 새로 만들어 10개를 표시하라. 조합을 나타내는 카테고리의 값은 임의로 정한다.

In [ ]:
# 성별 코드 1자리와 연대 코드 2자리를 연결한 성연령 코드를 생성한다.

df_customer[c("customer_id", "gender_cd", "birth_day", "age")] %>%
    mutate(era = trunc(age / 10) * 10) %>%
    mutate(era = case_when(
        era < 60 ~ formatC(era, width = 2,flag = "0"),
        era >= 60 ~　formatC(60, width = 2,flag = "0"))) %>%
    mutate(gender_era = paste(gender_cd, era, sep = "")) %>%
    slice(1:10)

customer_id,gender_cd,birth_day,age,era,gender_era
<chr>,<chr>,<date>,<int>,<chr>,<chr>
CS021313000114,1,1981-04-29,37,30,130
CS037613000071,9,1952-04-01,66,60,960
CS031415000172,1,1976-10-04,42,40,140
CS028811000001,1,1933-03-27,86,60,160
CS001215000145,1,1995-03-29,24,20,120
CS020401000016,0,1974-09-15,44,40,040
CS015414000103,1,1977-08-09,41,40,140
CS029403000008,0,1973-08-17,45,40,040
CS015804000004,0,1931-05-02,87,60,060


---
> R-058: 고객 데이터(df_customer)의 성별 코드(gender_cd)를 더미 변수로 만들어 고객 ID(customer_id)와 함께 10건 표시하라.

In [ ]:
# 코드 예시 1 (모든 코드 값을 항목화)
dummy_gender_model <- dummyVars(~gender_cd, 
                                data = df_customer, fullRank = FALSE)

dummy_gender <- predict(dummy_gender_model, df_customer)

head(cbind(df_customer["customer_id"], dummy_gender), 10)

,customer_id,gender_cd0,gender_cd1,gender_cd9
,<chr>,<dbl>,<dbl>,<dbl>
1,CS021313000114,0,1,0
2,CS037613000071,0,0,1
3,CS031415000172,0,1,0
4,CS028811000001,0,1,0
5,CS001215000145,0,1,0
6,CS020401000016,1,0,0
7,CS015414000103,0,1,0
8,CS029403000008,1,0,0
9,CS015804000004,1,0,0


In [ ]:
# 코드 예시 2 (fullRank=TRUE로 설정하여 항목을 하나 깎을 수 있음)
dummy_gender_model <- dummyVars(~gender_cd, 
                                data = df_customer, fullRank = TRUE)

dummy_gender <- predict(dummy_gender_model, df_customer)

head(cbind(df_customer["customer_id"], dummy_gender), 3)

,customer_id,gender_cd1,gender_cd9
,<chr>,<dbl>,<dbl>
1,CS021313000114,1,0
2,CS037613000071,0,1
3,CS031415000172,1,0


---
> R-059: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 고객 ID(customer_id)별로 합산하고, 매출 금액 합계를 평균 0, 표준편차 1로 표준화하여 고객 ID, 매출 금액 합계와 함께 10건씩 표시하라. 표준화에 사용하는 표준편차는 분산 제곱근 또는 불균형 분산 제곱근 중 어느 것이든 상관없다. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

In [ ]:
# R의 scale을 사용하기 때문에 불균등분산의 제곱근에 의한 표준편차로 표준화됨
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(std_amount = scale(sum_amount, 
                              center = TRUE, scale = TRUE)) %>%
    slice(1:10)

customer_id,sum_amount,std_amount
<chr>,<int>,"<dbl[,1]>"
CS001113000004,1298,-0.4593502
CS001114000005,626,-0.7063478
CS001115000010,3044,0.1824025
CS001205000004,1988,-0.2057366
CS001205000006,3337,0.2900964
CS001211000025,456,-0.7688324
CS001212000027,448,-0.7717728
CS001212000031,296,-0.8276413
CS001212000046,228,-0.8526351


---
> R-060: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 고객 ID(customer_id)별로 합산하여 매출 금액 합계를 최소값 0, 최대값 1로 정규화하여 고객 ID, 매출 금액 합계와 함께 10건씩 표시한다. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

In [ ]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(
        scale_amount = scale(sum_amount, 
                             center = min(sum_amount), 
                             scale = max(sum_amount) - min(sum_amount)))%>%
    slice(1:10)

customer_id,sum_amount,scale_amount
<chr>,<int>,"<dbl[,1]>"
CS001113000004,1298,0.053354188
CS001114000005,626,0.024157108
CS001115000010,3044,0.129214460
CS001205000004,1988,0.083333333
CS001205000006,3337,0.141944734
CS001211000025,456,0.016770942
CS001212000027,448,0.016423358
CS001212000031,296,0.009819256
CS001212000046,228,0.006864790


---
> R-061: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 고객 ID(customer_id)별로 합산하고, 매출 금액 합계를 상수 대수화(하단 10)하여 고객 ID, 매출 금액 합계와 함께 10건씩 표시한다. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

In [ ]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(log_amount = log((sum_amount + 0.5), base = 10)) %>%
    slice(1:10)

customer_id,sum_amount,log_amount
<chr>,<int>,<dbl>
CS001113000004,1298,3.113442
CS001114000005,626,2.796921
CS001115000010,3044,3.483516
CS001205000004,1988,3.298526
CS001205000006,3337,3.523421
CS001211000025,456,2.659441
CS001212000027,448,2.651762
CS001212000031,296,2.472025
CS001212000046,228,2.358886


---
> R-062: 영수증 내역 데이터(df_receipt)의 매출금액(amount)을 고객ID(customer_id)별로 합산하고, 매출금액 합계를 자연대수화(하단 e)하여 고객ID, 매출금액 합계와 함께 10건씩 표시한다. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

In [ ]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(log_amount = log(sum_amount + 0.5)) %>%
    slice(1:10)

customer_id,sum_amount,log_amount
<chr>,<int>,<dbl>
CS001113000004,1298,7.168965
CS001114000005,626,6.440149
CS001115000010,3044,8.021092
CS001205000004,1988,7.595136
CS001205000006,3337,8.112977
CS001211000025,456,6.123589
CS001212000027,448,6.105909
CS001212000031,296,5.692047
CS001212000046,228,5.431536


---
> R-063: 상품 데이터(df_product)의 단가(unit_price)와 원가(unit_cost)로부터 각 상품의 이익액을 산출하고, 그 결과를 10건 표시하시오.

In [ ]:
df_product %>%
    mutate(unit_profit = unit_price - unit_cost) %>%
    slice(1:10)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_profit
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>
P040101001,04,0401,040101,198,149,49
P040101002,04,0401,040101,218,164,54
P040101003,04,0401,040101,230,173,57
P040101004,04,0401,040101,248,186,62
P040101005,04,0401,040101,268,201,67
P040101006,04,0401,040101,298,224,74
P040101007,04,0401,040101,338,254,84
P040101008,04,0401,040101,420,315,105
P040101009,04,0401,040101,498,374,124


---
> R-064: 상품 데이터(df_product)의 단가(unit_price)와 원가(unit_cost)에서 각 상품의 전체 평균 수익률을 계산하시오. 단, 단가와 원가에는 결손이 발생한다는 점에 유의하라.

In [ ]:
df_product %>%
    mutate(unit_profit_rate = (unit_price - unit_cost) / unit_price) %>%
    summarise(total_mean = mean(unit_profit_rate, na.rm = TRUE))

total_mean
<dbl>
0.2491139


---
> R-065: 상품 데이터(df_product)의 각 상품에 대해 수익률이 30%가 되는 새로운 단가를 구하시오. 단, 1원 미만은 반올림한다. 그리고 10개의 결과를 표시하고, 수익률이 대략 30% 정도인 것을 확인하라. 단, 단가(unit_price)와 원가(unit_cost)에는 적자가 발생한다는 점에 유의하라.

In [ ]:
df_product[c("product_cd", "unit_price", "unit_cost")] %>%
    mutate(new_price = trunc(unit_cost / 0.7)) %>%
    mutate(new_profit_rate = (new_price - unit_cost)/ new_price) %>%
    slice(1:10)

product_cd,unit_price,unit_cost,new_price,new_profit_rate
<chr>,<int>,<int>,<dbl>,<dbl>
P040101001,198,149,212,0.2971698
P040101002,218,164,234,0.2991453
P040101003,230,173,247,0.2995951
P040101004,248,186,265,0.2981132
P040101005,268,201,287,0.2996516
P040101006,298,224,320,0.3000000
P040101007,338,254,362,0.2983425
P040101008,420,315,450,0.3000000
P040101009,498,374,534,0.2996255


---
> R-066: 상품 데이터(df_product)의 각 상품에 대해 수익률이 30%가 되는 새로운 단가를 구하시오. 이번에는 1원 미만은 반올림한다(반올림 또는 짝수로 반올림해도 무방하다). 그리고 10개의 결과를 표시하게 하고, 수익률이 대략 30% 정도인 것을 확인하라. 단, 단가(unit_price)와 원가(unit_cost)에는 결손이 발생한다는 점에 유의한다.

In [ ]:
df_product[c("product_cd", "unit_price", "unit_cost")] %>%
    mutate(new_price = round(unit_cost / 0.7)) %>%
    mutate(new_profit_rate = (new_price - unit_cost) / new_price) %>%
    slice(1:10)

product_cd,unit_price,unit_cost,new_price,new_profit_rate
<chr>,<int>,<int>,<dbl>,<dbl>
P040101001,198,149,213,0.3004695
P040101002,218,164,234,0.2991453
P040101003,230,173,247,0.2995951
P040101004,248,186,266,0.3007519
P040101005,268,201,287,0.2996516
P040101006,298,224,320,0.3000000
P040101007,338,254,363,0.3002755
P040101008,420,315,450,0.3000000
P040101009,498,374,534,0.2996255


---
> R-067: 상품 데이터(df_product)의 각 상품에 대해 수익률이 30%가 되는 새로운 단가를 구하시오. 이번에는 1원 미만은 반올림한다. 그리고 10개의 결과를 표시하고, 수익률이 대략 30% 정도인 것을 확인하라. 단, 단가(unit_price)와 원가(unit_cost)에는 적자가 발생하고 있다는 점에 유의하라.

In [ ]:
df_product[c("product_cd", "unit_price", "unit_cost")] %>%
    mutate(new_price = ceiling(unit_cost / 0.7)) %>%
    mutate(new_profit_rate = (new_price - unit_cost) / new_price) %>%
    slice(1:10)

product_cd,unit_price,unit_cost,new_price,new_profit_rate
<chr>,<int>,<int>,<dbl>,<dbl>
P040101001,198,149,213,0.3004695
P040101002,218,164,235,0.3021277
P040101003,230,173,248,0.3024194
P040101004,248,186,266,0.3007519
P040101005,268,201,288,0.3020833
P040101006,298,224,320,0.3000000
P040101007,338,254,363,0.3002755
P040101008,420,315,451,0.3015521
P040101009,498,374,535,0.3009346


---
> R-068: 상품 데이터(df_product)의 각 상품에 대해 소비세율 10%의 부가세 포함 금액을 구하고, 1원 미만의 단수는 절사하여 10개의 결과를 표시하시오. 단, 단가(unit_price)에는 결손이 발생한다는 점에 유의하라.

In [ ]:
head(cbind(product_cd = df_product$product_cd,
           unit_price = df_product$unit_price,
           tax_price = trunc(df_product$unit_price * 1.1)), 10)

product_cd,unit_price,tax_price
P040101001,198,217
P040101002,218,239
P040101003,230,253
P040101004,248,272
P040101005,268,294
P040101006,298,327
P040101007,338,371
P040101008,420,462
P040101009,498,547
P040101010,580,638


---
> R-069: 영수증 내역 데이터(df_receipt)와 상품 데이터(df_product)를 결합하여 고객별 전체 상품 판매금액 합계와 카테고리 대분류 코드(category_major_cd)가 "07"(병조림 통조림)인 상품의 판매금액 합계를 계산한 후, 양자의 비율을 구하시오. 추출 대상은 카테고리 대분류 코드 "07"(병조림 통조림)의 판매 실적이 있는 고객으로 한정하고, 결과를 10건만 표시한다.

In [ ]:
# 코드 예제 1
df_tmp_1 <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_all=sum(amount))

df_tmp_2 <- inner_join(df_receipt, 
                       df_product[c("product_cd","category_major_cd")], 
                       by="product_cd") %>% 
    filter(category_major_cd == "07") %>%
    group_by(customer_id) %>%
    summarise(sum_07 = sum(amount), .groups = "drop")

inner_join(df_tmp_1, df_tmp_2, by = "customer_id") %>%
    mutate(sales_rate = sum_07 / sum_all) %>%
    slice(1:10)

customer_id,sum_all,sum_07,sales_rate
<chr>,<int>,<int>,<dbl>
CS001113000004,1298,1298,1.0000000
CS001114000005,626,486,0.7763578
CS001115000010,3044,2694,0.8850197
CS001205000004,1988,346,0.1740443
CS001205000006,3337,2004,0.6005394
CS001212000027,448,200,0.4464286
CS001212000031,296,296,1.0000000
CS001212000046,228,108,0.4736842
CS001212000070,456,308,0.6754386


In [ ]:
# 코드 예시 2 (%>%로 모든 처리 연결)

inner_join(df_receipt, df_product[c("product_cd","category_major_cd")], 
           by="product_cd") %>%
    filter(category_major_cd == "07") %>%
    group_by(customer_id) %>%
    summarise(sum_07 = sum(amount), .groups = "drop") %>%
    inner_join(df_receipt, by="customer_id") %>%
    group_by(customer_id) %>%
    summarise(sum_all=sum(amount), 
              sum_07=max(sum_07), .groups = "drop") %>%
    mutate(sales_rate = sum_07 / sum_all) %>%
    slice(1:10)

customer_id,sum_all,sum_07,sales_rate
<chr>,<int>,<int>,<dbl>
CS001113000004,1298,1298,1.0000000
CS001114000005,626,486,0.7763578
CS001115000010,3044,2694,0.8850197
CS001205000004,1988,346,0.1740443
CS001205000006,3337,2004,0.6005394
CS001212000027,448,200,0.4464286
CS001212000031,296,296,1.0000000
CS001212000046,228,108,0.4736842
CS001212000070,456,308,0.6754386


---
> R-070: 영수증 내역 데이터(df_receipt)의 매출일(sales_ymd)에 대해 고객 데이터(df_customer)의 회원가입일(application_date)로부터의 경과일수를 계산하여 고객 ID(customer_id), 매출일, 회원가입일과 함께 10건을 표시하라(sales_ymd는 수치, application_date는 문자열로 데이터를 보관하고 있다는 점에 유의).

In [ ]:
df_receipt[c("customer_id", "sales_ymd")] %>%
    distinct(.,.keep_all=TRUE) %>%
    inner_join(df_customer[c("customer_id","application_date")], 
                            by="customer_id") %>%
    mutate(elapsed_days = as.integer(
                        strptime(as.character(sales_ymd), "%Y%m%d")
                                - strptime(application_date, "%Y%m%d"))) %>%
    select(customer_id, sales_ymd, application_date, elapsed_days) %>%
    slice(1:10)

customer_id,sales_ymd,application_date,elapsed_days
<chr>,<int>,<chr>,<int>
CS006214000001,20181103,20150201,1371
CS008415000097,20181118,20150322,1337
CS028414000014,20170712,20150711,732
CS025415000050,20180821,20160131,933
CS003515000195,20190605,20150306,1552
CS024514000042,20181205,20151010,1152
CS040415000178,20190922,20150627,1548
CS027514000015,20191010,20151101,1439
CS025415000134,20190918,20150720,1521


---
> R-071: 영수증 내역 데이터(df_receipt)의 매출일(sales_ymd)에 대해 고객 데이터(df_customer)의 회원가입일(application_date)로부터의 경과 개월 수를 계산하여 고객 ID(customer_id), 매출일, 회원가입일과 함께 10건 표시 (sales_ymd는 숫자, application_date는 문자열로 데이터를 보관하는 점에 유의) 1개월 미만은 반올림한다.

In [ ]:
df_receipt[c("customer_id", "sales_ymd")] %>%
    distinct(., .keep_all = TRUE) %>%
    inner_join(df_customer[c("customer_id", "application_date")], by = "customer_id") %>%
    mutate(elapsed_months = 
           trunc(time_length(
               interval(
                   strptime(application_date, "%Y%m%d"),
                   strptime(as.character(sales_ymd), "%Y%m%d")
               ), "month"))) %>%
    select(customer_id, sales_ymd, application_date, elapsed_months) %>%
    slice(1:10)

customer_id,sales_ymd,application_date,elapsed_months
<chr>,<int>,<chr>,<dbl>
CS006214000001,20181103,20150201,45
CS008415000097,20181118,20150322,43
CS028414000014,20170712,20150711,24
CS025415000050,20180821,20160131,30
CS003515000195,20190605,20150306,50
CS024514000042,20181205,20151010,37
CS040415000178,20190922,20150627,50
CS027514000015,20191010,20151101,47
CS025415000134,20190918,20150720,49


---
> R-072: 영수증 내역 데이터(df_receipt)의 매출일(df_customer)에 대해 고객 데이터(df_customer)의 회원가입 신청일(application_date)로부터의 경과년수를 계산하여 고객 ID(customer_id), 매출일, 회원가입 신청일과 함께 10건 (sales_ymd는 수치, application_date는 문자열로 데이터를 보관하고 있다는 점에 유의) 1년 미만은 반올림한다.

In [ ]:
df_receipt[c("customer_id", "sales_ymd")] %>%
    distinct(., .keep_all = TRUE) %>%
    inner_join(df_customer[c("customer_id", "application_date")], by = "customer_id") %>%
    mutate(elapsed_years = 
           trunc(time_length(interval(
               strptime(application_date, "%Y%m%d"),
               strptime(as.character(sales_ymd), "%Y%m%d")), "year")))%>%
    select(customer_id, sales_ymd, application_date, elapsed_years) %>%
    slice(1:10)

customer_id,sales_ymd,application_date,elapsed_years
<chr>,<int>,<chr>,<dbl>
CS006214000001,20181103,20150201,3
CS008415000097,20181118,20150322,3
CS028414000014,20170712,20150711,2
CS025415000050,20180821,20160131,2
CS003515000195,20190605,20150306,4
CS024514000042,20181205,20151010,3
CS040415000178,20190922,20150627,4
CS027514000015,20191010,20151101,3
CS025415000134,20190918,20150720,4


---
> R-073: 영수증 내역 데이터(df_receipt)의 매출일(sales_ymd)에 대해 고객 데이터(df_customer)의 회원가입일(application_date)로부터의 에포크 초 단위의 경과 시간을 계산하여 고객 ID(customer_id), 매출일, 회원가입일과 함께 10건을 표시한다. 과 함께 10건을 표시하라(단, sales_ymd는 수치, application_date는 문자열로 데이터를 보유하고 있다는 점에 유의). 단, 시간 정보는 보유하지 않으므로 각 날짜는 0시 0분 0초로 표시한다.

In [ ]:
df_receipt[c("customer_id", "sales_ymd")] %>%
    distinct(., .keep_all = TRUE) %>%
    inner_join(df_customer[c("customer_id","application_date")],
               by="customer_id") %>%
    mutate(elapsed_epoch = 
           as.numeric(strptime(as.character(sales_ymd), "%Y%m%d")) 
            - as.numeric(strptime(application_date, "%Y%m%d"))) %>%
    select(customer_id, sales_ymd, application_date, elapsed_epoch) %>%
    slice(1:10)

customer_id,sales_ymd,application_date,elapsed_epoch
<chr>,<int>,<chr>,<dbl>
CS006214000001,20181103,20150201,118454400
CS008415000097,20181118,20150322,115516800
CS028414000014,20170712,20150711,63244800
CS025415000050,20180821,20160131,80611200
CS003515000195,20190605,20150306,134092800
CS024514000042,20181205,20151010,99532800
CS040415000178,20190922,20150627,133747200
CS027514000015,20191010,20151101,124329600
CS025415000134,20190918,20150720,131414400


---
> R-074: 영수증 내역 데이터(df_receipt)의 매출일(sales_ymd)에 대해 해당 주 월요일부터의 경과일수를 계산하여 매출일, 직전 월요일까지 10건씩 표시하라(sales_ymd는 수치로 데이터를 보관하고 있다는 점에 유의).

In [ ]:
df_receipt["sales_ymd"] %>%
    # 아래에서는 시작일이 일요일이기 때문에 하루 전으로 시프트하여 일요일을 전주 마지막 날로 변환하고 있다.
    # 그 상태에서 floor_date를 하면 한 주의 시작일로 전주 일요일을 얻을 수 있기 때문이다,
    # 시작일을 월요일로 만들기 위해 1을 더하고 있다.
    mutate(monday = as.Date(floor_date(
        strptime(as.character(sales_ymd), "%Y%m%d") - 1 , 
                    unit = "week")) + 1) %>%
    mutate(elapsed_days = as.integer(
           as.Date(
               strptime(as.character(sales_ymd), "%Y%m%d")) - monday)) %>%
    select(sales_ymd, elapsed_days, monday) %>%
    slice(1:10)

sales_ymd,elapsed_days,monday
<int>,<int>,<date>
20181103,5,2018-10-29
20181118,6,2018-11-12
20170712,2,2017-07-10
20190205,1,2019-02-04
20180821,1,2018-08-20
20190605,2,2019-06-03
20181205,2,2018-12-03
20190922,6,2019-09-16
20170504,3,2017-05-01


---
> R-075: 고객 데이터(df_customer)에서 무작위로 1%의 데이터를 추출하여 맨 앞부터 10개를 표시하라.

In [ ]:
head(sample_frac(tbl = df_customer, size = 0.01), 10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS001612000311,岡本 美佳,1,女性,1951-03-21,68,211-0015,神奈川県川崎市中原区北谷町**********,S13001,20171118,0-00000000-0
2,CS018415000209,西村 由美子,1,女性,1974-11-26,44,203-0051,東京都東久留米市小山**********,S13018,20151001,C-20101026-D
3,CS020415000109,岡村 里奈,1,女性,1969-02-02,50,173-0001,東京都板橋区本町**********,S13020,20150207,A-20100922-D
4,CS040403000046,辻 獅童,0,男性,1977-01-23,42,226-0016,神奈川県横浜市緑区霧が丘**********,S14040,20150813,0-00000000-0
5,CS044313000008,河原 育子,1,女性,1985-06-03,33,144-0056,東京都大田区西六郷**********,S13044,20161204,0-00000000-0
6,CS038502000025,小柳 ケンイチ,0,男性,1963-06-01,55,134-0015,東京都江戸川区西瑞江**********,S13038,20151214,0-00000000-0
7,CS034402000064,川越 明,0,男性,1972-11-27,46,213-0031,神奈川県川崎市高津区宇奈根**********,S14034,20170913,0-00000000-0
8,CS015303000005,寺西 一徳,0,男性,1987-01-16,32,135-0016,東京都江東区東陽**********,S13015,20150223,0-00000000-0
9,CS029411000007,森永 璃子,1,女性,1976-12-01,42,134-0085,東京都江戸川区南葛西**********,S12029,20150709,5-20101007-9


---
> R-076: 고객 데이터(df_customer)에서 성별 코드(gender_cd)의 비율에 따라 무작위로 10%의 데이터를 층화 추출하고, 성별 코드별로 건수를 집계하라.

In [ ]:
set.seed(71)

df_customer %>%
    group_by(gender_cd) %>%
    sample_frac(0.1) %>%
    summarise(customer_num = n(), .groups = "drop")

gender_cd,customer_num
<chr>,<int>
0,298
1,1792
9,107


---
> R-077: 영수증 명세서 데이터(df_receipt)의 매출 금액을 고객 단위로 합산하고, 합산한 매출 금액의 편차를 추출하라. 단, 이상값은 매출금액 합계를 로그화한 후 평균과 표준편차를 계산하여 그 평균에서 3σ 이상 벗어난 것으로 한다(자연대수, 상용대수 모두 가능). 결과는 10건 표시하라.

In [ ]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(log_sum_amount = log(sum_amount + 0.5)) %>%
    filter(abs(log_sum_amount - mean(log_sum_amount)) 
                                / sd(log_sum_amount) > 3) %>%
    slice(1:10)

customer_id,sum_amount,log_sum_amount
<chr>,<int>,<dbl>
ZZ000000000000,12395003,16.3328


---
> R-078: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 고객 단위로 합산하고, 합산된 매출 금액의 외곽값을 추출한다. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다. 여기서 이상값은 1사분위와 3사분위의 차이인 IQR을 이용하여 '1사분위수 -1.5×IQR' 이하 또는 '3사분위수+1.5×IQR'을 초과하는 것으로 한다. 결과는 10건 표시한다.

In [ ]:
df_receipt %>%
    group_by(customer_id) %>%
    filter(!grepl("^Z", customer_id)) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    filter(
        sum_amount < quantile(sum_amount)[2] 
                     - 1.5 * (quantile(sum_amount)[4] - quantile(sum_amount)[2])
        |
        sum_amount > quantile(sum_amount)[4] 
                     + 1.5 * (quantile(sum_amount)[4] - quantile(sum_amount)[2])
    ) %>%
    slice(1:10)

customer_id,sum_amount
<chr>,<int>
CS001414000048,8584
CS001605000009,18925
CS002415000594,9568
CS004414000181,9584
CS005415000137,8734
CS006414000001,9156
CS006414000029,9179
CS006415000105,10042
CS006415000147,12723


---
> R-079: 상품 데이터(df_product)의 각 항목에 대해 결손 수를 확인하라.

In [ ]:
sapply(df_product, function(x) sum(is.na(x)))

product_cd  category_major_cd category_medium_cd  category_small_cd 
                 0                  0                  0                  0 
        unit_price          unit_cost 
                 7                  7

---
> R-080: 상품 데이터(df_product) 중 어느 한 항목에 결손이 발생한 레코드를 모두 삭제한 새로운 상품 데이터를 생성한다. 또한, 삭제 전후의 건수를 표시하고, 079에서 확인한 건수만큼 감소한 것도 확인해야 한다.

In [ ]:
df_product_1 <- na.omit(df_product)

paste("삭제전", nrow(df_product))
paste("삭제후:", nrow(df_product_1))

[1] "削除前: 10030"

[1] "削除後: 10023"

---
> R-081: 단가(unit_price)와 원가(unit_cost)의 결손값에 대해 각각의 평균값으로 보완한 새로운 상품 데이터를 작성하시오. 단, 평균값은 1원 미만은 반올림한다(반올림 또는 짝수로 반올림해도 무방하다). 보완 실시 후 각 항목에 대해 결손이 발생하지 않았는지도 확인해야 한다.

In [ ]:
price_mean <- round(mean(df_product$unit_price, na.rm = TRUE))

cost_mean <- round(mean(df_product$unit_cost, na.rm = TRUE))

df_product_2 <- df_product %>%
    replace_na(list(unit_price = price_mean, unit_cost = cost_mean))
    
sapply(df_product_2, function(x) sum(is.na(x)))

product_cd  category_major_cd category_medium_cd  category_small_cd 
                 0                  0                  0                  0 
        unit_price          unit_cost 
                 0                  0

---
> R-082: 단가(unit_price)와 원가(unit_cost)의 결손값에 대해 각각의 중앙값으로 보완한 새로운 상품 데이터를 작성하시오. 단, 중앙값은 1원 미만은 반올림한다(반올림 또는 짝수로 반올림해도 무방하다). 보완 실시 후 각 항목에 대해 결손이 발생하지 않았는지도 확인해야 한다.

In [ ]:
price_median <- round(median(df_product$unit_price, na.rm = TRUE))

cost_median <- round(median(df_product$unit_cost, na.rm = TRUE))

df_product_3 <- df_product %>%
    replace_na(list(unit_price = price_median, unit_cost = cost_median))
    
sapply(df_product_3, function(x) sum(is.na(x)))

product_cd  category_major_cd category_medium_cd  category_small_cd 
                 0                  0                  0                  0 
        unit_price          unit_cost 
                 0                  0

---
> R-083: 단가(unit_price)와 원가(unit_cost)의 결손값에 대해 각 상품의 카테고리 소분류 코드(category_small_cd)별로 산출한 중간값으로 보완한 새로운 상품 데이터를 작성한다. 단, 중앙값은 1원 미만은 반올림한다(반올림 또는 짝수로 반올림해도 무방하다). 보완 실시 후 각 항목에 대해 결손이 발생하지 않았는지도 확인해야 한다.

In [ ]:
df_product_4 <- df_product %>%
    group_by(category_small_cd) %>%
    summarise(price_median = round(median(unit_price, na.rm = TRUE)), 
              cost_median = round(median(unit_cost, na.rm = TRUE)),
              .groups = "drop") %>%
    inner_join(df_product, by = "category_small_cd") %>%
    mutate(unit_price = ifelse(is.na(unit_price), price_median, unit_price),
           unit_cost = ifelse(is.na(unit_cost), cost_median, unit_cost))

sapply(df_product_4, function(x) sum(is.na(x)))

category_small_cd       price_median        cost_median         product_cd 
                 0                  0                  0                  0 
 category_major_cd category_medium_cd         unit_price          unit_cost 
                 0                  0                  0                  0

---
> R-084: 고객 데이터(df_customer)의 전체 고객에 대해 전체 기간의 매출 금액에서 2019년 매출 금액이 차지하는 비율을 계산하여 새로운 데이터를 생성한다. 단, 매출 실적이 없는 경우 0으로 처리한다. 그리고 계산한 비율이 0을 초과하는 것을 추출하여 결과를 10건씩 표시하라. 또한, 작성된 데이터에 결손이 없는지 확인하라.

In [ ]:
df_receipt_2019 <- df_receipt %>%
                    filter(20190101 <= sales_ymd & sales_ymd <= 20191231) %>%
                    group_by(customer_id) %>%
                    summarise(amount_2019 = sum(amount), .groups = "drop")

df_receipt_all <- df_receipt %>%
                    group_by(customer_id) %>%
                    summarise(amount_all = sum(amount), .groups = "drop")

df_sales_rate <- left_join(df_customer["customer_id"],
                           df_receipt_2019, by = "customer_id") %>%
                    left_join(df_receipt_all, by = "customer_id") %>%
                    replace_na(list(amount_2019 = 0, amount_all = 0)) %>%
                    mutate(amount_rate = ifelse(amount_all == 0, 0,
                                                amount_2019 / amount_all))

df_sales_rate %>%
    filter(amount_rate > 0) %>%
    slice(1:10)

customer_id,amount_2019,amount_all,amount_rate
<chr>,<dbl>,<dbl>,<dbl>
CS031415000172,2971,5088,0.58392296
CS015414000103,874,3122,0.27994875
CS011215000048,248,3444,0.07200929
CS029415000023,3767,5167,0.72904974
CS035415000029,5823,7504,0.77598614
CS023513000066,208,771,0.26977951
CS035513000134,463,1565,0.29584665
CS001515000263,216,216,1.00000000
CS006415000279,229,229,1.00000000


In [ ]:
sapply(df_receipt_2019, function(x) sum(is.na(x)))

customer_id amount_2019 
          0           0

---
> R-085: 고객 데이터(df_customer)의 모든 고객에 대해 우편번호(postal_cd)를 이용하여 지오코드 데이터(df_geocode)를 연결하여 새로운 고객 데이터를 생성한다. 단, 하나의 우편번호(postal_cd)에 여러 개의 경도(longitude), 위도(latitude) 정보가 연결된 경우에는 경도(longitude), 위도(latitude)의 평균값을 산출하여 사용해야 한다. 또한, 생성된 결과를 확인하기 위해 10개의 결과를 표시한다.

In [ ]:
df_geocode_1 <- df_geocode[c("postal_cd", "longitude" ,"latitude")] %>%
                group_by(postal_cd) %>%
                summarise(m_longiture = mean(longitude),
                          m_latitude = mean(latitude), .groups = "drop")

df_customer_1 <- inner_join(df_customer, df_geocode_1, by = "postal_cd")

head(df_customer_1, 10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,m_longiture,m_latitude
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.3178,35.41358
2,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.8350,35.67193
3,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,139.6897,35.67374
4,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,139.4836,35.39125
5,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,139.7078,35.54084
6,CS020401000016,宮下 達士,0,男性,1974-09-15,44,174-0065,東京都板橋区若木**********,S13020,20150225,0-00000000-0,139.6724,35.77073
7,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B,139.8360,35.67818
8,CS029403000008,釈 人志,0,男性,1973-08-17,45,279-0003,千葉県浦安市海楽**********,S12029,20150515,0-00000000-0,139.9047,35.65422
9,CS015804000004,松谷 米蔵,0,男性,1931-05-02,87,136-0073,東京都江東区北砂**********,S13015,20150607,0-00000000-0,139.8360,35.67818


---
> R-086: 085에서 생성한 위도경도별 고객 데이터에 대해 회원 신청 매장 코드(application_store_cd)를 키로 매장 데이터(df_store)와 결합하라. 그리고 신청 매장의 위도(latitude)-경도 정보(longitude)와 고객 주소(address)의 위도-경도를 이용하여 신청 매장과 고객 주소의 거리(단위: km)를 구하고, 고객 ID(customer_id), 고객 주소(address), 매장 주소(address)와 함께 표시하라. 과 함께 표시하라. 계산식은 아래의 간단한 식을 사용하되, 그 외 정밀도가 높은 방식을 이용한 라이브러리를 사용해도 무방하다. 결과는 10건을 표시한다.

$$
\begin{aligned}
& \mbox{위도(라디안)} : \phi \\ \\
& \mbox{경도(라디안)}：\lambda \\
& \mbox{거리}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2 \\)
& \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 - \lambda_2))
\end{aligned}
$$

In [ ]:
calc_distance <- function(x1, y1, x2, y2) {
    distance <- 6371 * acos(
                        sin(x1 * pi / 180) 
                        * sin(x2 * pi / 180) 
                        + cos(x1 * pi / 180) 
                        * cos(x2 * pi / 180) 
                        * cos((y1 * pi / 180) - (y2 * pi / 180 )))
    print(class(distance)) # Check the CLASS of the calculation results just to be sure.
    return(distance)
}

inner_join(df_customer_1, df_store, 
           by = c("application_store_cd" = "store_cd")) %>%
    rename(customer_address = address.x, store_address = address.y) %>%
    mutate(distance = 
           calc_distance(m_latitude, m_longiture, latitude, longitude)) %>%
    select(customer_id, customer_address, store_address, distance) %>%
    slice(1:10)

[1] "numeric"


customer_id,customer_address,store_address,distance
<chr>,<chr>,<chr>,<dbl>
CS021313000114,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.3944087
CS037613000071,東京都江東区南砂**********,東京都江東区南砂一丁目,1.4511822
CS031415000172,東京都渋谷区代々木**********,東京都渋谷区初台二丁目,0.4117335
CS028811000001,神奈川県横浜市泉区和泉町**********,神奈川県横浜市瀬谷区二ツ橋町,8.0651960
CS001215000145,東京都大田区仲六郷**********,東京都大田区仲六郷二丁目,1.2684210
CS020401000016,東京都板橋区若木**********,東京都北区十条仲原三丁目,4.1859046
CS015414000103,東京都江東区北砂**********,東京都江東区南砂二丁目,1.4496734
CS029403000008,千葉県浦安市海楽**********,千葉県浦安市東野一丁目,0.8048581
CS015804000004,東京都江東区北砂**********,東京都江東区南砂二丁目,1.4496734


---
> R-087: 고객 데이터(df_customer)에는 다른 매장에서의 신청 등으로 동일 고객이 여러 개 등록되어 있다. 이름(customer_name)과 우편번호(postal_cd)가 같은 고객은 동일 고객으로 간주하여 1고객 1레코드가 되도록 이름을 붙인 명목 고객 데이터를 생성하고, 고객 데이터 개수, 명목 고객 데이터 개수, 중복 횟수를 계산하라. 단, 동일 고객에 대해서는 매출 금액 합계가 가장 높은 것을 남기고, 매출 금액 합계가 동일하거나 매출 실적이 없는 고객에 대해서는 고객 ID(customer_id)의 번호가 작은 것을 남긴다.

In [ ]:
df_sales_amount <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

df_customer_u <- left_join(df_customer, df_sales_amount, by = "customer_id") %>%
    mutate(sum_amount = ifelse(is.na(sum_amount), 0, sum_amount)) %>%
    arrange(desc(sum_amount), customer_id) %>%
    distinct(customer_name, postal_cd, .keep_all = TRUE) 

print(paste(
    "df_customer_cnt:", nrow(df_customer),
    "df_customer_u_cnt:", nrow(df_customer_u),
    "diff:", nrow(df_customer) - nrow(df_customer_u)))

[1] "df_customer_cnt: 21971 df_customer_u_cnt: 21941 diff: 30"


---
> R-088: 087에서 생성한 데이터를 바탕으로 고객 데이터에 통합명칭 ID를 부여한 데이터를 생성한다. 단, 통합명칭 ID는 아래의 사양으로 부여한다.
>  중복되지 않은 고객
> - 중복되지 않은 고객 : 고객 ID(customer_id) 설정
> - 중복되는 고객: 이전 설문에서 추출한 레코드의 고객 ID를 설정한다.
>고객 ID의 고유 개수 
> 고객 ID의 고유 건수와 통합명칭 ID의 고유 건수 차이도 확인해야 한다.

In [ ]:
df_customer_n = inner_join(df_customer, 
                           df_customer_u[c("customer_id", 
                                           "customer_name", 
                                           "postal_cd")],
                           by = c("customer_name", "postal_cd")) %>% 
                    rename(customer_id = customer_id.x, 
                           integration_id = customer_id.y)

customer_id_cnt <- length(unique(df_customer_n$customer_id))
integration_id_cnt <- length(unique(df_customer_n$integration_id))

print(paste("수의 차이", customer_id_cnt - integration_id_cnt))

[1] "ID数の差 30"


---
> R-089: 예측 모델 구축을 위해 판매 실적이 있는 고객을 학습용 데이터와 테스트용 데이터로 나누고 싶다. 각각 8:2의 비율로 무작위로 데이터를 분할하라.

In [ ]:
set.seed(71)

#Example using rsample initial_split
df_sales_customer <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    filter(sum_amount > 0)

df_tmp <- inner_join(df_customer, df_sales_customer, by = "customer_id")

split <- initial_split(df_tmp, prop = 0.8)

df_customer_train <- training(split)
df_customer_test <- testing(split)

# Confirmation
print(paste("학습 데이터 비율:", nrow(df_customer_train) / nrow(df_tmp)))
print(paste("테스트 데이터 비율:", nrow(df_customer_test) / nrow(df_tmp)))

[1] "学習データ割合: 0.799903684083795"
[1] "テストデータ割合: 0.200096315916205"


---
> R-090: 영수증 내역 데이터(df_receipt)는 2017년 1월 1일부터 2019년 10월 31일까지의 데이터를 가지고 있다. 매출 금액(amount)을 월별로 집계하여 학습용 12개월, 테스트용 6개월의 시계열 모델 구축용 데이터 3세트를 생성한다.

In [ ]:
#caret의 createTimeSlices를 사용한 예시
df_ts_amount <- df_receipt %>%
    group_by(substr(as.character(sales_ymd), 1, 6)) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

colnames(df_ts_amount) <- c("sales_ym", "sales_amount")

timeSlice <- createTimeSlices(df_ts_amount$sales_ym, 
                              initialWindow = 12, 
                              horizon = 6, 
                              skip = 5, 
                              fixedWindow = TRUE)

df_train_1 <- df_ts_amount[timeSlice$train[[1]],]
df_train_2 <- df_ts_amount[timeSlice$train[[2]],]
df_train_3 <- df_ts_amount[timeSlice$train[[3]],]

df_test_1 <- df_ts_amount[timeSlice$test[[1]],]
df_test_2 <- df_ts_amount[timeSlice$test[[2]],]
df_test_3 <- df_ts_amount[timeSlice$test[[3]],]

In [ ]:
# df_train_2와 df_train_3의 표시는 생략 df_train_1
df_train_1

sales_ym,sales_amount
<chr>,<int>
201701,902056
201702,764413
201703,962945
201704,847566
201705,884010
201706,894242
201707,959205
201708,954836
201709,902037


In [ ]:
# df_test_2와 df_test_3의 표시 생략 df_test_1
df_test_1

sales_ym,sales_amount
<chr>,<int>
201801,944509
201802,864128
201803,946588
201804,937099
201805,1004438
201806,1012329


---
> R-091: 고객 데이터(df_customer)의 각 고객에 대해 매출 실적이 있는 고객 수와 매출 실적이 없는 고객 수가 1:1이 되도록 언더샘플링으로 추출하라.

In [ ]:
#receipts 패키지 사용 예시
df_sales_amount <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    right_join(df_customer, by = "customer_id") %>%
    mutate(is_buy_flag = factor(ifelse(is.na(sum_amount), 0, 1)))

df_down_sampling <- df_sales_amount %>% 
    recipe() %>% 
    step_downsample(is_buy_flag, seed = 71) %>%
    prep() %>% 
    juice()

df_down_sampling %>%
    group_by(is_buy_flag) %>%
    summarise(cnt = n(), .groups = "drop")

is_buy_flag,cnt
<fct>,<int>
0,8306
1,8306


---
> R-092: 고객 데이터(df_customer)의 성별을 제3정규형으로 정규화하라.

In [ ]:
df_gender_std = unique(df_customer[c("gender_cd", "gender")])

df_customer_std = df_customer[, colnames(df_customer) != "gender"]

In [ ]:
# 데이터 내용 확인
head(df_customer_std, n = 3)

,customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS021313000114,大野 あや子,1,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
2,CS037613000071,六角 雅彦,9,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS031415000172,宇多田 貴美子,1,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C


In [ ]:
# 데이터 내용 확인
head(df_gender_std, n = 3)

,gender_cd,gender
,<chr>,<chr>
1,1,女性
2,9,不明
6,0,男性


---
> R-093: 상품 데이터(df_product)는 각 카테고리의 코드 값만 보유하고 카테고리 이름은 보유하지 않는다. 카테고리 데이터(df_category)와 결합하여 비정규화하여 카테고리 이름을 보유한 새로운 상품 데이터를 생성한다.

In [ ]:
df_product_full <- inner_join(df_product, 
                              df_category[c("category_small_cd", 
                                            "category_major_name",
                                            "category_medium_name",
                                            "category_small_name")], 
                              by = "category_small_cd")

# 데이터 내용 확인
head(df_product_full, n = 3)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>
1,P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
2,P040101002,04,0401,040101,218,164,惣菜,御飯類,弁当類
3,P040101003,04,0401,040101,230,173,惣菜,御飯類,弁当類


---
> R-094: 093에서 생성된 카테고리 이름을 가진 제품 데이터를 다음 사양으로 파일로 정리하라.
>
> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |CSV(쉼표로 구분된 값)|있음|UTF-8|
>
> 파일 출력 대상의 경로는 다음과 같아야 합니다.
> 
> |출력 대상|
> |:--:|
> |. /data|

In [ ]:
# 답안 파일의 위치가 문제 파일과 다르기 때문에 경로가 '. /data'로 되어 있는 점 참고
write.csv(df_product_full, "../data/R_df_product_full_UTF-8_header.csv", 
          row.names=FALSE,
          fileEncoding = "UTF-8")

---
> R-095: 093에서 생성된 카테고리 이름을 가진 제품 데이터를 다음 사양으로 파일로 정리합니다.
>
> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |CSV(쉼표로 구분된 값)|있음|CP932|
>
> 파일 출력 대상의 경로는 다음과 같아야 합니다.
> 
> |출력 대상|
> |:--:|
> |. /data|

In [ ]:
# 답안 파일의 위치가 문제 파일과 다르기 때문에 경로가 '. /data'로 되어 있는 점 참고
write.csv(df_product_full, "../data/R_df_product_full_CP932_header.csv", 
          row.names = FALSE,
          fileEncoding = "CP932")

---
> R-096: 093에서 생성된 카테고리 이름을 가진 제품 데이터를 다음 사양으로 파일로 정리하라.

> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |CSV(쉼표로 구분된 값)|없음|UTF-8|
>
> 파일 출력 대상의 경로는 다음과 같아야 합니다.
> 
> |출력 대상|
> |:--:|
> |. /data|

In [ ]:
# 답안 파일의 위치가 문제 파일과 다르기 때문에 경로가 '. /data'로 되어 있는 점 참고
write.table(df_product_full, "../data/R_df_product_full_UTF-8_noh.csv", 
            row.names = FALSE,
            col.names = FALSE, sep = ",", fileEncoding = "UTF-8")

---
> R-097: 094에서 만든 다음 형식의 파일을 로드하고 세 개의 데이터 항목을 표시한 다음 올바르게 가져왔는지 확인합니다.
> 
> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |CSV(쉼표로 구분된 값)|있음|UTF-8|

In [ ]:
# 답안 파일의 위치가 문제 파일과 다르기 때문에 경로가 '. /data'로 되어 있는 점 참고
c_class <- c(NA, "character", "character", "character", NA, NA, NA, NA, NA)

df_product_full <- read.csv("../data/R_df_product_full_UTF-8_header.csv", 
                           colClasses = c_class,
                           fileEncoding = "UTF-8")

head(df_product_full, 3)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>
1,P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
2,P040101002,04,0401,040101,218,164,惣菜,御飯類,弁当類
3,P040101003,04,0401,040101,230,173,惣菜,御飯類,弁当類


---
> R-098: 096에서 만든 다음 형식의 파일을 로드하고 세 개의 데이터 항목을 표시한 다음 올바르게 가져왔는지 확인합니다.
> 
> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |CSV(쉼표로 구분된 값)|있음|UTF-8|

In [ ]:
# 코드 예시 1 (나중에 항목 이름 붙이기)
# 답안 파일의 위치가 문제 파일과 다르기 때문에 경로가 '. /data'로 되어 있는 점 참고
c_class <- c(NA, "character", "character", "character", NA, NA, NA, NA, NA)

df_product_full <- read.csv("../data/R_df_product_full_UTF-8_noh.csv", 
                           colClasses = c_class,
                           header = FALSE, fileEncoding = "UTF-8")

colnames(df_product_full) <- c("product_cd", "category_major_cd",
                               "category_medium_cd", "category_small_cd",
                               "unit_price", "unit_cost", 
                               "category_major_name", "category_medium_name", 
                               "category_small_name")

head(df_product_full, 3)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>
1,P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
2,P040101002,04,0401,040101,218,164,惣菜,御飯類,弁当類
3,P040101003,04,0401,040101,230,173,惣菜,御飯類,弁当類


In [ ]:
# 코드 예제 2(항목 이름을 먼저 정의)
# 답안 파일의 위치가 문제 파일과 다르기 때문에 경로가 '. /data'로 되어 있는 점 참고
c_names <- c("product_cd", "category_major_cd", "category_medium_cd",
             "category_small_cd", "unit_price","unit_cost",
             "category_major_name", "category_medium_name",
             "category_small_name")

c_class <- c(NA, "character", "character", "character", NA, NA, NA, NA, NA)

df_product_full <- read.csv("../data/R_df_product_full_UTF-8_noh.csv", 
                           col.names = c_names,
                           colClasses = c_class,
                           header = FALSE, fileEncoding = "UTF-8")

head(df_product_full, 3)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>
1,P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
2,P040101002,04,0401,040101,218,164,惣菜,御飯類,弁当類
3,P040101003,04,0401,040101,230,173,惣菜,御飯類,弁当類


---
> R-099: 093에서 생성된 카테고리 이름을 가진 제품 데이터를 다음 사양으로 파일로 정리합니다.

> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |TSV(탭으로 구분된 값)|있음|UTF-8|
>
> 파일 출력 대상의 경로는 다음과 같아야 합니다.
> 
> |출력 대상|
> |:--:|
> |. /data|


In [ ]:
# 답안 파일의 위치가 문제 파일과 다르기 때문에 경로가 '. /data'로 되어 있는 점 참고
write.table(df_product_full, "../data/R_df_product_full_UTF-8_header.tsv", 
            row.names = FALSE, sep = "\t", 
            fileEncoding = "UTF-8")

---
> R-100: 093에서 생성된 카테고리 이름을 가진 제품 데이터를 다음 사양의 파일로 정리합니다.
> 
> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |TSV(탭으로 구분된 값)|있음|UTF-8|

In [ ]:
# 답안 파일의 위치가 문제 파일과 다르기 때문에 경로가 '. /data'로 되어 있는 점 참고
c_class <- c(NA, "character", "character", "character", NA, NA, NA, NA, NA)

df_product_tmp <- read.table("../data/R_df_product_full_UTF-8_header.tsv", 
                             colClasses = c_class,
                             header = TRUE, fileEncoding = "UTF-8")

head(df_product_tmp,3)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>
1,P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
2,P040101002,04,0401,040101,218,164,惣菜,御飯類,弁当類
3,P040101003,04,0401,040101,230,173,惣菜,御飯類,弁当類


# 이것으로 100번 노크는 끝났습니다. 수고하셨습니다!